# State of the Business Analysis

## Case Description

You are a BA working for a startup that has one credit product called **Contigo**. Depending on risk, the customer will receive:
1. Contigo A, a 1,000 peso loan to be paid in full after one month with 15% interest. This product is given to high risk customers.
2. Contigo B, a 2,000 peso loan to be paid in full after one month with 10% interest. This product is given to low risk customers.

Past debt delinquency (DQ) rates have determined the interest rates for the Contigo product. However, it is estimated that the monthly DQ rate for Contigo A will be 13.5% and 8% for Contigo B, throughout this year.

Ignoring the worsening of DQ rates, this startup has an ambitious growth plan for the year. At month 1,
the company had a total of 5,000 Contigo A customers, expected to grow at a rate of 25% month over
month. Contigo B has the largest share of customers, 10,000 in total. But with a very competitive market
for premium loans, Contigo B is expected to grow at a 10% month over month rate throughout the year.

The operation of this startup scales proportionately to its customer base, with negligible fixed expenses. The current operative monthly cost per account sits at 30 pesos.

The startup is concerned that the change in DQ rates and the aggressive expansion may reduce profitability in the following months, so they asked you to make a forecast on the net income of its operation (ignoring taxes) in the next 12 months.

### Loan Types Diagram

<img 
    src="https://i.imgur.com/B7NvEKb.png"
    height=300
    width=650
/>

In [1355]:
# Contigo A loan features
contigo_a_features = {
    "loan_amt":1000, "loan_term":1, "interest_rate":0.15,
    "delinquency_rate":0.135, "initial_num_accounts":5000,
    "account_growth_mom":0.25, "op_cost_per_acct":30
}

# Contigo B loan features
contigo_b_features = {
    "loan_amt":2000, "loan_term":1, "interest_rate":0.10,
    "delinquency_rate":0.08, "initial_num_accounts":10000,
    "account_growth_mom":0.10, "op_cost_per_acct":30
}

## Answering Questions

### Question 1

> What factors may be against the profitability of the startup? What should we look at in an analysis of the net income for the following months?

#### Factors Against Profitability

1. **Delinquency Rates**: One of the main factors that can negatively impact the profitability of the startup is high delinquency rates of their products. The startup should aim to maintain a wide gap between interest rates and delinquency rates, ensuring that the delinquency rate never exceeds the interest rate.
2. **Customer Growth Rate**: Adding more customers increases the risk exposure, so the startup should pursue aggressive growth strategies when delinquency rates are low and less aggressive growth strategies when delinquency rates are high.
3. **Operating Expenses**: Another factor that can play against profitability is the expense of account management, such as spending too much on collection efforts due to unnecessary or expensive communication channels with customers or higher prices of required account management software.
4. **Policy Rates**: A higher policy rate could reduce the demand for loans and potentially reduce the interest revenue for the startup. It could also negatively affect some customers as they will need to pay more for their loans, increasing the potential loss of principal and interest during each period. The startup could hedge against rate hikes by using interest rate swaps or interest rate futures.

Overall, the startup should aim to diversify their income to prevent dependence on a single product for their income.

### Question 2

> Is this startup going to be profitable during the next 12 months? 
>
> What are the major trends driving changes in net income? 
>
> Provide a model for net income and forecast it for the next 12 months. 
>
>> **Note**: Losses come from the total debt not being repaid, and revenue comes from collected interest. We will only consider operation costs as expenses.

In [1356]:
from net_income_model import financial_results

Based on the initial loan features for Contigo A and Contigo B loans, the startup is not profitable due to high delinquency rates for both types of loans:

In [1357]:
financial_results(contigo_a_features, contigo_b_features)

loan_type,Contigo A,Contigo B,Total
interest_revenue,"35,166,531.75","39,346,560.00","74,513,091.75"
delinquency_loss,"36,583,000.00","34,204,000.00","70,787,000.00"
operative_costs,"8,130,990.00","6,415,200.00","14,546,190.00"
net_income,"-9,547,458.25","-1,272,640.00","-10,820,098.25"


The startup could reach profitability in the following scenarios:
- `Delinquency Rate <= 10.40%` for **Contigo A**, with a net income of **$113,845.20**, if everything else stays the same.

In [1366]:
contigo_a_features["delinquency_rate"] = 0.1043

financial_results(contigo_a_features, contigo_b_features)

loan_type,Contigo A,Contigo B,Total
interest_revenue,"36,414,638.71","39,346,560.00","75,761,198.72"
delinquency_loss,"28,262,000.00","34,204,000.00","62,466,000.00"
operative_costs,"8,130,990.00","6,415,200.00","14,546,190.00"
net_income,"21,648.71","-1,272,640.00","-1,250,991.29"


- `Delinquency Rate <= 7.7%` for Contigo B, with a net income of **$137,664.00**, if everything else stays the same.

In [1374]:
contigo_b_features["delinquency_rate"] = 0.077

financial_results(contigo_a_features, contigo_b_features)

loan_type,Contigo A,Contigo B,Total
interest_revenue,"36,414,638.71","39,474,864.00","75,889,502.72"
delinquency_loss,"28,262,000.00","32,922,000.00","61,184,000.00"
operative_costs,"8,130,990.00","6,415,200.00","14,546,190.00"
net_income,"21,648.71","137,664.00","159,312.72"


- `Interest Rate >= 16.80%` for Contigo A, with a net income of **$93,185.56**, if everything else stays the same.

- `Interest Rate >= 10.40%` for Contigo B, with a net income of **$301,222.40**, if everything else stays the same.

#### Total Results

> **Question 3**: What changes are required to sustain the business not only throughout the year but in the long run?
>
> Build a proposal of changes to main startup KPIs to achieve this goal. 
>
>Discuss what difficulties the startup may find in applying your recommendation.

To sustain the business not only throughout the year but in the long run, the startup should focus on:
- Improving the credit risk assessment process to reduce the DQ rates.
- Diversify the revenue stream and reduce reliance on a single product.

KPI:
- Interest-to-Delinquency:

    * $\frac {InterestRate}{DelinquencyRate} \ge 1.24$ for Contigo A

    * $\frac {InterestRate}{DelinquencyRate} \ge 1.30$ for Contigo B